In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [8]:
cloth0_dir_names = []
cloth1_dir_names = []
cloth2_dir_names = []

for dir in sorted(os.listdir("/media/ExtraDrive3/tactilecloth/bagfiles/norub/0cloth_norub_auto")):
    cloth0_dir_names.append("/media/ExtraDrive3/tactilecloth/bagfiles/norub/0cloth_norub_auto/"+dir)

for dir in sorted(os.listdir("/media/ExtraDrive3/tactilecloth/bagfiles/norub/1cloth_norub_auto")):
    cloth1_dir_names.append("/media/ExtraDrive3/tactilecloth/bagfiles/norub/1cloth_norub_auto/"+dir)

for dir in sorted(os.listdir("/media/ExtraDrive3/tactilecloth/bagfiles/norub/2cloth_norub_auto")):
    cloth2_dir_names.append("/media/ExtraDrive3/tactilecloth/bagfiles/norub/2cloth_norub_auto/"+dir)

In [21]:
def get_data(dir_name):
    file_name = dir_name+"/reskin_data.csv"
    data = np.loadtxt(file_name, delimiter = ",")
    data = data[:,:-1]
    return data

def get_data_XY(dir_name):
    file_name = dir_name+"/reskin_data.csv"
    data = np.loadtxt(file_name, delimiter = ",")
    X = data[:,:-2]
    Y = np.reshape(data[:,-2],(-1,1))
    return X,Y

def plot_magnetometer(data):
    fig, axes = plt.subplots(nrows = 5, ncols = 1, sharex = True)
    names = {0:"Center", 1: "Top", 2: "Right", 3: "Bottom", 4: "Left"}
    for i in range(5):
        axes[i].plot(data[:,i*3], 'r', label="Bx")
        axes[i].plot(data[:,i*3+1], 'g', label = "By")
        axes[i].plot(data[:,i*3+2], 'b', label = "Bz")
        axes[i].set_title(names[i])
        pass
    lines, labels = fig.axes[-2].get_legend_handles_labels()
    fig.legend(lines, labels, loc = 'lower right', fontsize =12.0)
    fig.tight_layout(pad=0.5)
    fig.text(0.04, 0.5, 'Magnetometer Data', va='center', rotation='vertical', fontsize = 14.0)
    fig.set_size_inches(20, 11.3)  
    plt.show()

def plot_diff(data):
    fig, axes = plt.subplots(nrows = 5, ncols = 1, sharex = True)
    names = {0:"Center", 1: "Top", 2: "Right", 3: "Bottom", 4: "Left"}
    dx = 0.001
    for i in range(5):
        axes[i].plot(np.diff(data[:,i*3])/dx, 'r', label="Bx")
        axes[i].plot(np.diff(data[:,i*3+1])/dx, 'g', label = "By")
        axes[i].plot(np.diff(data[:,i*3+2])/dx, 'b', label = "Bz")
        axes[i].set_title(names[i])
        pass
    lines, labels = fig.axes[-2].get_legend_handles_labels()
    fig.legend(lines, labels, loc = 'lower right', fontsize =12.0)
    fig.tight_layout(pad=0.5)
    fig.text(0.04, 0.5, 'Magnetometer Data', va='center', rotation='vertical', fontsize = 14.0)
    fig.set_size_inches(20, 11.3)  
    plt.show()

def plot_norm(data):
    fig = plt.figure()
    sqdata = np.square(data)
    norm_data = np.abs(np.diff(np.mean(sqdata, axis = 1)))
    plt.plot(norm_data, 'r', label="norm")
    fig.set_size_inches(20, 11.3)  
    plt.show()

def plot_magnetometers_with_label(data):
    fig, axes = plt.subplots(nrows = 6, ncols = 1, sharex = True)
    names = {0:"Center", 1: "Top", 2: "Right", 3: "Bottom", 4: "Left"}
    axes[-1].plot(data[:,-1], 'black', label="Contact Data")
    for i in range(5):
        axes[i].plot(data[:,i*3], 'r', label="Bx")
        axes[i].plot(data[:,i*3+1], 'g', label = "By")
        axes[i].plot(data[:,i*3+2], 'b', label = "Bz")
        axes[i].set_title(names[i])
        pass
    lines, labels = fig.axes[-2].get_legend_handles_labels()
    fig.legend(lines, labels, loc = 'lower right', fontsize =12.0)
    fig.tight_layout(pad=0.5)
    fig.text(0.04, 0.5, 'Magnetometer Data', va='center', rotation='vertical', fontsize = 14.0)
    fig.set_size_inches(20, 11.3)  
    plt.show()

In [10]:
num_train = 10
num_test = 5
train_dir_names = cloth0_dir_names[0:num_train] + cloth1_dir_names[0:num_train] + cloth2_dir_names[0:num_train]
test_dir_names = cloth0_dir_names[num_train:] + cloth1_dir_names[num_train:] + cloth2_dir_names[num_train:]

train_dir_names_str = ""
for dir_name in train_dir_names:
    train_dir_names_str = train_dir_names_str + dir_name +"\n"

test_dir_names_str = ""
for dir_name in test_dir_names:
    test_dir_names_str = test_dir_names_str + dir_name +"\n"

with open("test_dir_names_norub.txt", "w") as fn:
    fn.write(test_dir_names_str)
    
with open("train_dir_names_norub.txt", "w") as fn:
    fn.write(train_dir_names_str)

In [22]:
X = []
Y = []
x_test = []
y_test = []
for dir_name in train_dir_names:
    x,y = get_data_XY(dir_name)
    X.append(x)
    Y.append(y)
X = np.vstack(X)
Y = np.vstack(Y)
print(X.shape, Y.shape)

for dir_name in test_dir_names:
    x,y = get_data_XY(dir_name)
    x_test.append(x)
    y_test.append(y)
x_test = np.vstack(x_test)
y_test = np.vstack(y_test)
print(x_test.shape, y_test.shape)

((23952, 15), (23952, 1))
((11992, 15), (11992, 1))


In [23]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import neighbors
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)
x_test = scaler.transform(x_test)

In [24]:
clf = neighbors.KNeighborsClassifier(10, weights="distance")
print(X.shape, Y.ravel().shape)
print(Y)
clf.fit(X, Y.ravel())


((23952, 15), (23952,))
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='distance')

In [25]:
y_pred = clf.predict(x_test)
from sklearn.metrics import balanced_accuracy_score
score = balanced_accuracy_score(y_test, y_pred)
print("balanced accuracy: ",score)
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(x_test)
best_cf = confusion_matrix(y_test, y_pred)
print(best_cf)

('balanced accuracy: ', 0.9989621297507749)
[[5539    2    3    0]
 [   4 2154    0    0]
 [   1    0 2132    0]
 [   2    0    0 2155]]
